<a href="https://colab.research.google.com/github/kimdonggyu2008/music_generation/blob/main/genetic_algorithm_chord_progression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from dataclasses import dataclass

import music21

In [ ]:
@dataclass(frozen=True)
class MelodyData: #멜로디 데이터의 정보 저장
  notes: list #노트(음)
  duration: int = None # 길이
  number_of_bars: int = None  # 전체 마디 갯수


  def __post__init(self):
    object.__setattr__(
        self,"duration",sum(duration for _, duration in self.notes)
    )
    object.__setattr__(self,"number_of_bars",self.duration//4)


class GeneticMelodyHarmonizer:
  def __init__( #유전 알고리즘 멜로디, 코드 합성
      self,
      melody_data,
      chords,
      population_size,
      mutation_rate,
      fitness_evaluator,
  ):
    self.melody_data=melody_data #멜로디
    self.chords=chords #코드
    self.mutation_rate=mutation_rate #변이 확률
    self.population_size=population_size # 총 크기
    self.fitness_evaluator=fitness_evaluator #어울리는지 확인(?)
    self._population=[]#총 크기는 비워놓고 생성

  def generate(self, generations=1000):
    self._population=self._initialise_population()#초기 값으로 생성(?)
    for _ in range(generations):# 노트 총 횟수만큼 반복
      parents=self._select_parents() #현재 이전의 노트
      new_population=self._create_new_population(parents) # 이전 노트를 가지고 현재 노트를 만들어냄
      self._population=new_population #현재 노트로 설정
    best_chord_sequence=(
      self.fitness_evaluator.get_chord_sequence_with_highest_fitness( #지금까지 나온 노트 중 가장 나은 것을 고르기
        self._population
      )
    )
    return best_chord_sequence

  def _initialise_population(self):
    return [
        self._generate_random_chord_sequence() #랜덤한 코드를 현재 위치에 생성함
        for _ in range(self.population_size)
    ]

  def _generate_random_chord_sequence(self):
    return [
        random.choice(self.chords) #여러 개의 코드 중에서 하나를 고름
        for _ in range(self.melody_data.number_of_bars) #각 바에 대해서 코드를 하나씩 골라서 지정함
    ]

  def _select_parents(self): # ???
    fitness_values=[
        self.fitness_evaluator.evaluate(seq) for seq in self._population #각 바의 코드에 대해서 얼마나 어울리는지 확인(?)
    ]
    return random.choices( # ????
        self._population,weights=fitness_values,k=self.population_size
    )

  def _create_new_population(self,parents): #크로스 오버를 통한 그다음 세대 생성
    new_population=[]
    for i in range(0,self.population_size,2):
      child1,child2=self._crossover( #차일드 1, 차일드 2는 각각 패런트 i, 패런트 i+1의 값을 가지고 섞음
          parents[i],parents[i+1]#한번은 앞쪽, 뒤쪽, 나머지 한번은 뒤쪽, 앞쪽으로 합함
      ), self._crossover(parents[i+1],parents[i])
      child1=self._mutate(child1) #각각 낮은 확률로 돌연변이
      child2=self._mutate(child2)
      new_population.extend([child1,child2]) #차일드 1, 2를 묶어서 해당 집합을 증가시킴
    return new_population #패런트를 활용해서 생성된 전체 차일드들을 반환함

  def _crossover(self,parent1,parent2): #앞뒤를 섞어서 차일드를 만듦
    cut_index_random.randint(1,len(parent1)-1)
    return parent1[:cut_index]+parent2[cut_index:]

  def _mutate(self,chord_sequence): #일정 확률로 돌연변이
    if random.random()<self.mutation_rate: #확률이 걸리면
      mutation_index=random.randint(0,len(chord_sequence)-1) #랜덤으로 하나 고름
      chord_sequence[mutation_index]=random.choice(self.chords) #바뀔수도 안 바뀔수도 있음
    return chord_sequence

class FitnessEvaluator: #평가기
  def __init__(
      self,melody_data,chord_mappings,weights,preferred_transitions
  ):
    self.melody_data=melody_data #멜로디
    self.chord_mappings=chord_mappings #생성된 코드
    self.weights=weights #가중치
    self.preffered_transitions=preferred_transitions #선호되는 코드 진행


  # 여기 다시 보기

  def get_chord_sequence_with_highest_fitness(self,chord_sequences):# ???
    return max(chord_sequences,key=self.evaluate) #가장 높은 fitness값을 가지는 값 반환

  def evaluate(self,chord_sequence):
    return sum(
        self.weights[func]*getattr(self,f"_{func}")(chord_sequence)#가중치 계산
        for func in self.weights
    )

  def _chord_melody_congruence(self,chord_sequence): #코드, 멜로디의 일치도 평가
    score,melody_index=0,0
    for chord in chord_sequence:
      bar_duration=0
      while bar_duration<4 and melody_index < len( #4박마다 계산, 모든 노트를 다 계산
          self.melody_data.notes
      ):
        pitch, duration=self.melody_data.notes[melody_index] #현재 노트의 음, 길이 가져옴
        if pitch[0] in self.chord_mappings[chord]: #마디의 첫 음이 코드의 구성음인 경우
          score+=duration # 해당 4박을 처리함
        bar_duration+=duration # 4박이 처리되면 한마디(4박)을 증가시킴
        melody_index+=1# 멜로디 인덱스 증가시킴
    return score/self.melody_data.duration #총 박자수 반환(?)


  #여기도 다시 보기
  def _chord_variety(self,chord_sequence): #???
    unique_chords=len(set(chord_sequence))
    total_chords=len(self.chord_mappings)
    return unique_chords/total_chords

  def _harmonic_flow(self,chord_sequence):
    score=0
    for i in range(len(chord_sequence)-1):#총 코드 갯수 만큼
      next_chord=chord_sequence[i+1] #다음 코드는 현재 바로 다음 코드임
      if next_chord in self.preferred_transitions[chord_sequence[i]]: #선호되는 코드 진행에 다음 코드가 존재할 경우
        score+=1 # 점수 증가
    return score/(len(chord_sequence)-1) #어울리는 마디 갯수/ 총 마디 갯수

  def _functional_harmony(self,chord_sequence):
    score=0
    if chord_sequence[0] in ["C","Am"]: #첫 코드가 해당 모양을 가질시 추가점수
      score+=1
    if chord_sequence[-1] in ["C"]:  #마지막 마디가 c일시 추가 점수
      score+=1
    if "F" in chord_sequence and "G" in chord_sequence: #f와 g가 둘 다 있을 시 추가점수
      score+=1
    return score/3

def create_score(melody,chord_sequence,chord_mappings): #악보생성
  score=music21.stream.Score()

  melody_part=music21.stream.Part()
  for note_name,duration in melody:
    melody_note=music21.note.Note(note_name,quarterLength=duration)
    melody_part.append(melody_note)

  chord_part=music21.stream.Part()
  current_duration=0
  for chord_name in chord_sequence:
    chord_notes_list=chord_mappings.get(chord_name,[])
    chord_notes=music21.chord.Chord(
        chord_notes_list,quarterLength=4
    )
    chord_notes.offset=current_duration
    chord_part.append(chord_notes)
    current_duration+=4

  score.append(melody_part)
  score.append(chord_part)

  return score

def main():
  twinkle_twinkle_melody = [ #주어진 멜로디 데이터
        ("C5", 1),
        ("C5", 1),
        ("G5", 1),
        ("G5", 1),
        ("A5", 1),
        ("A5", 1),
        ("G5", 2),  # Twinkle, twinkle, little star,
        ("F5", 1),
        ("F5", 1),
        ("E5", 1),
        ("E5", 1),
        ("D5", 1),
        ("D5", 1),
        ("C5", 2),  # How I wonder what you are!
        ("G5", 1),
        ("G5", 1),
        ("F5", 1),
        ("F5", 1),
        ("E5", 1),
        ("E5", 1),
        ("D5", 2),  # Up above the world so high,
        ("G5", 1),
        ("G5", 1),
        ("F5", 1),
        ("F5", 1),
        ("E5", 1),
        ("E5", 1),
        ("D5", 2),  # Like a diamond in the sky.
        ("C5", 1),
        ("C5", 1),
        ("G5", 1),
        ("G5", 1),
        ("A5", 1),
        ("A5", 1),
        ("G5", 2),  # Twinkle, twinkle, little star,
        ("F5", 1),
        ("F5", 1),
        ("E5", 1),
        ("E5", 1),
        ("D5", 1),
        ("D5", 1),
        ("C5", 2)  # How I wonder what you are!
    ]
  weights = {
        "chord_melody_congruence": 0.4,
        "chord_variety": 0.1,
        "harmonic_flow": 0.3,
        "functional_harmony": 0.2
    }
  chord_mappings = { #각 키에 해당하는 코드 구성음
        "C": ["C", "E", "G"],
        "Dm": ["D", "F", "A"],
        "Em": ["E", "G", "B"],
        "F": ["F", "A", "C"],
        "G": ["G", "B", "D"],
        "Am": ["A", "C", "E"],
        "Bdim": ["B", "D", "F"]
    }
  preferred_transitions = { #코드 진행 추천구성
        "C": ["G", "Am", "F"],
        "Dm": ["G", "Am"],
        "Em": ["Am", "F", "C"],
        "F": ["C", "G"],
        "G": ["Am", "C"],
        "Am": ["Dm", "Em", "F", "C"],
        "Bdim": ["F", "Am"]
    }

  melody_data=MelodyData(twinkle_twinkle_melody)#멜로디 데이터 선언

  fitness_evaluator=FitnessEvaluator(
      melody_data=melody_data,
      weights=weights,
      chord_mappings=chord_mappings,
      preferred_transitions=preferred_transitions,
  )
  harmonizer=GeneticMelodyHarmonizer(
      melody_data=melody_data,
      chords=list(chord_mappings.keys()),
      population_size=100,
      mutation_rate=0.05,
      fitness_evaluator=fitness_evaluator,
  )

  generated_chords=harmonizer.generate(generations=1000)

  music21_score=create_score(
      twinkle_twinkle_melody,generated_chords,chord_mappings
  )
  music21_score.show()

if __name__=="__main__":
  main()


TypeError: MelodyData.__init__() missing 2 required positional arguments: 'duration' and 'number_of_bars'

In [ ]:
import random
from dataclasses import dataclass

import music21


@dataclass(frozen=True)
class MelodyData:
    """
    A data class representing the data of a melody.

    This class encapsulates the details of a melody including its notes, total
    duration, and the number of bars. The notes are represented as a list of
    tuples, with each tuple containing a pitch and its duration. The total
    duration and the number of bars are computed based on the notes provided.

    Attributes:
        notes (list of tuples): List of tuples representing the melody's notes.
            Each tuple is in the format (pitch, duration).
        duration (int): Total duration of the melody, computed from notes.
        number_of_bars (int): Total number of bars in the melody, computed from
            the duration assuming a 4/4 time signature.

    Methods:
        __post_init__: A method called after the data class initialization to
            calculate and set the duration and number of bars based on the
            provided notes.
    """

    notes: list
    duration: int = None  # Computed attribute
    number_of_bars: int = None  # Computed attribute

    def __post_init__(self):
        object.__setattr__(self, "number_of_bars", self.duration // 4)


class GeneticMelodyHarmonizer:
    """
    Generates chord accompaniments for a given melody using a genetic algorithm.
    It evolves a population of chord sequences to find one that best fits the
    melody based on a fitness function.

    Attributes:
        melody_data (MusicData): Data containing melody information.
        chords (list): Available chords for generating sequences.
        population_size (int): Size of the chord sequence population.
        mutation_rate (float): Probability of mutation in the genetic algorithm.
        fitness_evaluator (FitnessEvaluator): Instance used to assess fitness.
    """

    def __init__(
        self,
        melody_data,
        chords,
        population_size,
        mutation_rate,
        fitness_evaluator,
    ):
        """
        Initializes the generator with melody data, chords, population size,
        mutation rate, and a fitness evaluator.

        Parameters:
            melody_data (MusicData): Melody information.
            chords (list): Available chords.
            population_size (int): Size of population in the algorithm.
            mutation_rate (float): Mutation probability per chord.
            fitness_evaluator (FitnessEvaluator): Evaluator for chord fitness.
        """
        self.melody_data = melody_data
        self.chords = chords
        self.mutation_rate = mutation_rate
        self.population_size = population_size
        self.fitness_evaluator = fitness_evaluator
        self._population = []

    def generate(self, generations=1000):
        """
        Generates a chord sequence that harmonizes a melody using a genetic
        algorithm.

        Parameters:
            generations (int): Number of generations for evolution.

        Returns:
            best_chord_sequence (list): Harmonization with the highest fitness
                found in the last generation.
        """
        self._population = self._initialise_population()
        for _ in range(generations):
            parents = self._select_parents()
            new_population = self._create_new_population(parents)
            self._population = new_population
        best_chord_sequence = (
            self.fitness_evaluator.get_chord_sequence_with_highest_fitness(
                self._population
            )
        )
        return best_chord_sequence

    def _initialise_population(self):
        """
        Initializes population with random chord sequences.

        Returns:
            list: List of randomly generated chord sequences.
        """
        return [
            self._generate_random_chord_sequence()
            for _ in range(self.population_size)
        ]

    def _generate_random_chord_sequence(self):
        """
        Generate a random chord sequence with as many chords as the numbers
        of bars in the melody.

        Returns:
            list: List of randomly generated chords.
        """
        return [
            random.choice(self.chords)
            for _ in range(self.melody_data.number_of_bars)
        ]

    def _select_parents(self):
        """
        Selects parent sequences for breeding based on fitness.

        Returns:
            list: Selected parent chord sequences.
        """
        fitness_values = [
            self.fitness_evaluator.evaluate(seq) for seq in self._population
        ]
        return random.choices(
            self._population, weights=fitness_values, k=self.population_size
        )

    def _create_new_population(self, parents):
        """
        Generates a new population of chord sequences from the provided parents.

        This method creates a new generation of chord sequences using crossover
        and mutation operations. For each pair of parent chord sequences,
        it generates two children. Each child is the result of a crossover
        operation between the pair of parents, followed by a potential
        mutation. The new population is formed by collecting all these
        children.

        The method ensures that the new population size is equal to the
        predefined population size of the generator. It processes parents in
        pairs, and for each pair, two children are generated.

        Parameters:
            parents (list): A list of parent chord sequences from which to
                generate the new population.

        Returns:
            list: A new population of chord sequences, generated from the
                parents.

        Note:
            This method assumes an even population size and that the number of
            parents is equal to the predefined population size.
        """
        new_population = []
        for i in range(0, self.population_size, 2):
            child1, child2 = self._crossover(
                parents[i], parents[i + 1]
            ), self._crossover(parents[i + 1], parents[i])
            child1 = self._mutate(child1)
            child2 = self._mutate(child2)
            new_population.extend([child1, child2])
        return new_population

    def _crossover(self, parent1, parent2):
        """
        Combines two parent sequences into a new child sequence using one-point
        crossover.

        Parameters:
            parent1 (list): First parent chord sequence.
            parent2 (list): Second parent chord sequence.

        Returns:
            list: Resulting child chord sequence.
        """
        cut_index = random.randint(1, len(parent1) - 1)
        return parent1[:cut_index] + parent2[cut_index:]

    def _mutate(self, chord_sequence):
        """
        Mutates a chord in the sequence based on mutation rate.

        Parameters:
            chord_sequence (list): Chord sequence to mutate.

        Returns:
            list: Mutated chord sequence.
        """
        if random.random() < self.mutation_rate:
            mutation_index = random.randint(0, len(chord_sequence) - 1)
            chord_sequence[mutation_index] = random.choice(self.chords)
        return chord_sequence


class FitnessEvaluator:
    """
    Evaluates the fitness of a chord sequence based on various musical criteria.

    Attributes:
        melody (list): List of tuples representing notes as (pitch, duration).
        chords (dict): Dictionary of chords with their corresponding notes.
        weights (dict): Weights for different fitness evaluation functions.
        preferred_transitions (dict): Preferred chord transitions.
    """

    def __init__(
        self, melody_data, chord_mappings, weights, preferred_transitions
    ):
        """
        Initialize the FitnessEvaluator with melody, chords, weights, and
        preferred transitions.

        Parameters:
            melody_data (MelodyData): Melody information.
            chord_mappings (dict): Available chords mapped to their notes.
            weights (dict): Weights for each fitness evaluation function.
            preferred_transitions (dict): Preferred chord transitions.
        """
        self.melody_data = melody_data
        self.chord_mappings = chord_mappings
        self.weights = weights
        self.preferred_transitions = preferred_transitions

    def get_chord_sequence_with_highest_fitness(self, chord_sequences):
        """
        Returns the chord sequence with the highest fitness score.

        Parameters:
            chord_sequences (list): List of chord sequences to evaluate.

        Returns:
            list: Chord sequence with the highest fitness score.
        """
        return max(chord_sequences, key=self.evaluate)

    def evaluate(self, chord_sequence):
        """
        Evaluate the fitness of a given chord sequence.

        Parameters:
            chord_sequence (list): The chord sequence to evaluate.

        Returns:
            float: The overall fitness score of the chord sequence.
        """
        return sum(
            self.weights[func] * getattr(self, f"_{func}")(chord_sequence)
            for func in self.weights
        )

    def _chord_melody_congruence(self, chord_sequence):
        """
        Calculates the congruence between the chord sequence and the melody.
        This function assesses how well each chord in the sequence aligns
        with the corresponding segment of the melody. The alignment is
        measured by checking if the notes in the melody are present in the
        chords being played at the same time, rewarding sequences where the
        melody notes fit well with the chords.

        Parameters:
            chord_sequence (list): A list of chords to be evaluated against the
                melody.

        Returns:
            float: A score representing the degree of congruence between the
                chord sequence and the melody, normalized by the melody's
                duration.
        """
        score, melody_index = 0, 0
        for chord in chord_sequence:
            bar_duration = 0
            while bar_duration < 4 and melody_index < len(
                self.melody_data.notes
            ):
                pitch, duration = self.melody_data.notes[melody_index]
                if pitch[0] in self.chord_mappings[chord]:
                    score += duration
                bar_duration += duration
                melody_index += 1
        return score / self.melody_data.duration

    def _chord_variety(self, chord_sequence):
        """
        Evaluates the diversity of chords used in the sequence. This function
        calculates a score based on the number of unique chords present in the
        sequence compared to the total available chords. Higher variety in the
        chord sequence results in a higher score, promoting musical
        complexity and interest.

        Parameters:
            chord_sequence (list): The chord sequence to evaluate.

        Returns:
            float: A normalized score representing the variety of chords in the
                sequence relative to the total number of available chords.
        """
        unique_chords = len(set(chord_sequence))
        total_chords = len(self.chord_mappings)
        return unique_chords / total_chords

    def _harmonic_flow(self, chord_sequence):
        """
        Assesses the harmonic flow of the chord sequence by examining the
        transitions between successive chords. This function scores the
        sequence based on how frequently the chord transitions align with
        predefined preferred transitions. Smooth and musically pleasant
        transitions result in a higher score.

        Parameters:
            chord_sequence (list): The chord sequence to evaluate.

        Returns:
            float: A normalized score based on the frequency of preferred chord
                transitions in the sequence.
        """
        score = 0
        for i in range(len(chord_sequence) - 1):
            next_chord = chord_sequence[i + 1]
            if next_chord in self.preferred_transitions[chord_sequence[i]]:
                score += 1
        return score / (len(chord_sequence) - 1)

    def _functional_harmony(self, chord_sequence):
        """
        Evaluates the chord sequence based on principles of functional harmony.
        This function checks for the presence of key harmonic functions such as
        the tonic at the beginning and end of the sequence and the presence of
        subdominant and dominant chords. Adherence to these harmonic
        conventions is rewarded with a higher score.

        Parameters:
            chord_sequence (list): The chord sequence to evaluate.

        Returns:
            float: A score representing the extent to which the sequence
                adheres to traditional functional harmony, normalized by
                the number of checks performed.
        """
        score = 0
        if chord_sequence[0] in ["C", "Am"]:
            score += 1
        if chord_sequence[-1] in ["C"]:
            score += 1
        if "F" in chord_sequence and "G" in chord_sequence:
            score += 1
        return score / 3


def create_score(melody, chord_sequence, chord_mappings):
    """
    Create a music21 score with a given melody and chord sequence.

    Args:
        melody (list): A list of tuples representing notes in the format
            (note_name, duration).
        chord_sequence (list): A list of chord names.

    Returns:
        music21.stream.Score: A music score containing the melody and chord
            sequence.
    """
    # Create a Score object
    score = music21.stream.Score()

    # Create the melody part and add notes to it
    melody_part = music21.stream.Part()
    for note_name, duration in melody:
        melody_note = music21.note.Note(note_name, quarterLength=duration)
        melody_part.append(melody_note)

    # Create the chord part and add chords to it
    chord_part = music21.stream.Part()
    current_duration = 0  # Track the duration for chord placement

    for chord_name in chord_sequence:
        # Translate chord names to note lists
        chord_notes_list = chord_mappings.get(chord_name, [])
        # Create a music21 chord
        chord_notes = music21.chord.Chord(
            chord_notes_list, quarterLength=4
        )  # Assuming 4/4 time signature
        chord_notes.offset = current_duration
        chord_part.append(chord_notes)
        current_duration += 4  # Increase by 4 beats

    # Append parts to the score
    score.append(melody_part)
    score.append(chord_part)

    return score


def main():

    twinkle_twinkle_melody = [
        ("C5", 1),
        ("C5", 1),
        ("G5", 1),
        ("G5", 1),
        ("A5", 1),
        ("A5", 1),
        ("G5", 2),  # Twinkle, twinkle, little star,
        ("F5", 1),
        ("F5", 1),
        ("E5", 1),
        ("E5", 1),
        ("D5", 1),
        ("D5", 1),
        ("C5", 2),  # How I wonder what you are!
        ("G5", 1),
        ("G5", 1),
        ("F5", 1),
        ("F5", 1),
        ("E5", 1),
        ("E5", 1),
        ("D5", 2),  # Up above the world so high,
        ("G5", 1),
        ("G5", 1),
        ("F5", 1),
        ("F5", 1),
        ("E5", 1),
        ("E5", 1),
        ("D5", 2),  # Like a diamond in the sky.
        ("C5", 1),
        ("C5", 1),
        ("G5", 1),
        ("G5", 1),
        ("A5", 1),
        ("A5", 1),
        ("G5", 2),  # Twinkle, twinkle, little star,
        ("F5", 1),
        ("F5", 1),
        ("E5", 1),
        ("E5", 1),
        ("D5", 1),
        ("D5", 1),
        ("C5", 2)  # How I wonder what you are!
    ]
    weights = {
        "chord_melody_congruence": 0.4,
        "chord_variety": 0.1,
        "harmonic_flow": 0.3,
        "functional_harmony": 0.2
    }
    chord_mappings = {
        "C": ["C", "E", "G"],
        "Dm": ["D", "F", "A"],
        "Em": ["E", "G", "B"],
        "F": ["F", "A", "C"],
        "G": ["G", "B", "D"],
        "Am": ["A", "C", "E"],
        "Bdim": ["B", "D", "F"]
    }
    preferred_transitions = {
        "C": ["G", "Am", "F"],
        "Dm": ["G", "Am"],
        "Em": ["Am", "F", "C"],
        "F": ["C", "G"],
        "G": ["Am", "C"],
        "Am": ["Dm", "Em", "F", "C"],
        "Bdim": ["F", "Am"]
    }

    # Instantiate objects for generating harmonization
    melody_data = MelodyData(twinkle_twinkle_melody)
    fitness_evaluator = FitnessEvaluator(
        melody_data=melody_data,
        weights=weights,
        chord_mappings=chord_mappings,
        preferred_transitions=preferred_transitions,
    )
    harmonizer = GeneticMelodyHarmonizer(
        melody_data=melody_data,
        chords=list(chord_mappings.keys()),
        population_size=100,
        mutation_rate=0.05,
        fitness_evaluator=fitness_evaluator,
    )

    # Generate chords with genetic algorithm
    generated_chords = harmonizer.generate(generations=1000)

    # Render to music21 score and show it
    music21_score = create_score(
        twinkle_twinkle_melody, generated_chords, chord_mappings
    )
    music21_score.show()


if __name__ == "__main__":
    main()

TypeError: unsupported operand type(s) for //: 'NoneType' and 'int'